In [1]:
import numpy as np
import numpy.typing as npt
import matplotlib.pyplot as plt  # Biblioteca para gerar gráficos
import pandas as pd
from sklearn import metrics, model_selection
from scipy import stats
from scipy.spatial import distance
import math
import random


In [2]:

class Set:
    def __init__(self, dataset, features, output):
        self.dataset = dataset
        self.features = features
        self.output = output

    def get_n(self):
        return self.dataset.shape[0]

    def get_x(self):
        return self.dataset[:, self.features]
    
    def get_x_apply(self, func):
        return func(self.dataset[:, self.features])

    def set_x(self, new_x):
        self.x = new_x

    def get_y(self):
        return self.dataset[:, self.output]

    def get_X(self, func=None):
        if (func):
            return np.c_[np.ones(self.get_n()), func(self.get_x())]
        else:
            return np.c_[np.ones(self.get_n()), self.get_x()]


In [3]:
def is_true_positive(y, y_pred):
    return y_pred >= 1 and y >= 1

def is_false_positive(y, y_pred):
    return y_pred >= 1 and y <= 0

def is_true_negative(y, y_pred):
    return y_pred <= 0 and y <= 0

def is_false_negative(y, y_pred):
    return y_pred <= 0 and y >= 1

def confusion_matrix(y, y_pred):
    """ returns (tp, fp, tn, fn) """

    tp, fp, tn, fn = 0, 0, 0, 0
    for i, pred in enumerate(y_pred):
        tp += 1 if is_true_positive(y[i], pred) else 0
        fp += 1 if is_false_positive(y[i], pred) else 0
        tn += 1 if is_true_negative(y[i], pred) else 0
        fn += 1 if is_false_negative(y[i], pred) else 0
    return (tp, fp, tn, fn)

def accuracy(y, y_pred):
    tp, fp, tn, fn = confusion_matrix(y, y_pred)
    return (tp + tn) / (tp + fp + tn + fn)

def precision(y, y_pred):
    tp, fp, tn, fn = confusion_matrix(y, y_pred)
    return tp / (tp + fp)

def recall(y, y_pred):
    tp, fp, tn, fn = confusion_matrix(y, y_pred)
    return tp / (tp + fn)

def f1_score(y, y_pred):
    precision_ = precision(y, y_pred)
    recall_ = recall(y, y_pred)
    return 2 * (precision_ * recall_) / (precision_ + recall_)

def get_metrics(n_folds):
    return {
        "accuracy": np.zeros(n_folds),
        "precision": np.zeros(n_folds),
        "recall": np.zeros(n_folds),
        "f1_score": np.zeros(n_folds)
    }

def calculate_metrics(metrics, y_test, y_pred):
    metrics["accuracy"][i] = (accuracy(y_test, y_pred))
    metrics["precision"][i] = (precision(y_test, y_pred))
    metrics["recall"][i] = (recall(y_test, y_pred))
    metrics["f1_score"][i] = (f1_score(y_test, y_pred))

def print_metrics(metrics, name, n_folds):
    print("%i-fold cross validation com %s" % (n_folds, name))
    print("acurácia: %.8f +/- %.8f" % (metrics["accuracy"].mean(), metrics["accuracy"].std()))
    print("revocação: %.8f +/- %.8f" % (metrics["precision"].mean(), metrics["precision"].std()))
    print("precisão: %.8f +/- %.8f" % (metrics["recall"].mean(), metrics["recall"].std()))
    print("f1-score: %.8f +/- %.8f" % (metrics["f1_score"].mean(), metrics["f1_score"].std()))
    print("")


In [4]:
def k_fold_split(array, k = int):
    """realiza o split dos dados em k-folds"""
    shuffled_data = np.random.permutation(array)
    folds = np.array_split(shuffled_data, k)
    return folds

def k_fold_train_test(folds):
    """retorna um vetor com as configurações de treino e teste definidas pelo k-fold split

    returns (i, train, test)
    """
    results = []
    for i, fold in enumerate(folds):
        train = np.vstack([x for j, x in enumerate(folds) if j != i])
        test = fold
        results.append((i, train, test))
    return results

# Questão 1

**Observação**: Não esqueça de normalizar os dados em ambas as questões.

Considere o conjunto de dados disp onível em **quake.csv**, organizado em 2 colunas de atributos. Os dados referem-se a latitudes e longitudes de locais em que foram registrados terremotos. Maiores detalhes sobre os dados podem ser conferidos em https://www.openml.org/d/772.

In [5]:
data = np.genfromtxt('./californiabin.csv', delimiter=',')
np.random.seed(666)
features = np.arange(8)
labels = 8

a) Avalie o algoritmo K-médias com distância Euclidiana na tarefa de agrupamento para tais dados. O número de grupos deve ser escolhido entre os valores 4,5,6,...19,20 a partir do índice DB (Davies-Bouldin). Plote o melhor resultado de agrupamento obtido.

**Observação**: Em cada avaliação repita múltiplas vezes (por exemplo, 20) a execução do algoritmo K-médias, escolhendo a solução com melhor erro de reconstrução.

b) Repita o item anterior considerando a distância de Mahalanobis.

# Questão 2

Considere o conjunto de dados disponível em **penguins.csv**, organizado em 5 colunas, sendo 4 colunas de atributos e a última a classe do padrão. Os dados referem-se a medições anatômicas de pinguins da Antártida, classificados nas espécies Adelie, Chinstrap e Gentoo. Maiores detalhes sobre os dados podem ser conferidos em https://allisonhorst.github.io/palmerpenguins/.

a) Apresente a projeção em 2 dimensões dos padrões acima obtida pelo método PCA (análise dos componentes principais).

b) Ainda considerando o item anterior, calcule e mostre a variância explicada obtida quando a dimensão projetada é modificada (1,2,3 ou 4).